In [ ]:
# This project was originally developed in R for a machine learning course.
# Redeveloping in Python3 as a programming exercise.

In [19]:
import numpy as np
import pandas as pd
# import pandas_profiling # AttributeError: module 'pandas_profiling' has no attribute 'templates'

In [2]:
charity = pd.read_csv('charity.csv')

In [12]:
# Pause EDA until environmental issue with pandas_profiling can be worked out.
# Work from known facts about the data from previous project version.

charity.head(5)

,ID,reg1,reg2,reg3,reg4,home,chld,hinc,genf,wrat,...,npro,tgif,lgif,rgif,tdon,tlag,agif,donr,damt,part
0,1,0,0,1,0,1,1,4,1,8,...,20,81,81,19,17,6,21.05,0.0,0.0,train
1,2,0,0,1,0,1,2,4,0,8,...,95,156,16,17,19,3,13.26,1.0,15.0,train
2,3,0,0,1,0,1,1,5,1,8,...,64,86,15,10,22,8,17.37,NaN,NaN,test
3,4,0,0,0,0,1,1,4,0,8,...,51,56,18,7,14,7,9.59,NaN,NaN,test
4,5,0,0,1,0,1,0,4,1,4,...,85,132,15,10,10,6,12.07,1.0,17.0,valid


In [13]:
charity.shape

(8009, 24)

In [21]:
charity.train = charity.loc[charity['part'] == 'train']

In [22]:
charity.train.shape

(3984, 24)

In [23]:
charity.valid = charity.loc[charity['part'] == 'valid']

In [24]:
charity.valid.shape

(2018, 24)

In [25]:
charity.test = charity.loc[charity['part'] == 'test']

In [26]:
charity.test.shape

(2007, 24)